# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [26]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [27]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [28]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 '

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [29]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [30]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [31]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edwar

In [32]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [33]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'project page', 'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'project page', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'company page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'Twitter', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'Facebook', 'url': 'https://www.facebook.com/edward.donner.52'}]}

In [34]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [35]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 7 relevant links


{'links': [{'type': 'company homepage', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'patent',
   'url': 'https://patents.google.com/patent/US20210049536A1/'},
  {'type': 'LinkedIn profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'Twitter profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'Facebook profile',
   'url': 'https://www.facebook.com/edward.donner.52'},
  {'type': 'Nebula company site',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'}]}

In [36]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano


KeyboardInterrupt: 

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [37]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [38]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 5 relevant links


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
deepseek-ai/DeepSeek-OCR
Updated
1 day ago
•
488k
•
1.7k
PaddlePaddle/PaddleOCR-VL
Updated
about 7 hours ago
•
13.6k
•
1.03k
tencent/HunyuanWorld-Mirror
Updated
2 days ago
•
4.49k
•
306
Qwen/Qwen3-VL-8B-Instruct
Updated
9 days ago
•
216k
•
315
krea/krea-realtime-video
Updated
4 days ago
•
894
•
156
Browse 1M+ models
Spaces
Running
501
501
veo3.1-fast
🐨
Generate videos from text or images
Running
15.4k
15.4k
DeepSite v3
🐳
Generate any application by Vibe Coding
Running
431
431
Sora 2
📉
Generate videos from text or images
Running
on
Zero
195
195
DeepSeek OCR Demo
🆘
An interactive demo for the DeepSeek-OCR model.
Running
2.03k
2.03k

In [39]:
# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""


In [40]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [41]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 12 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\ndeepseek-ai/DeepSeek-OCR\nUpdated\n1 day ago\n•\n488k\n•\n1.7k\nPaddlePaddle/PaddleOCR-VL\nUpdated\nabout 7 hours ago\n•\n13.6k\n•\n1.03k\ntencent/HunyuanWorld-Mirror\nUpdated\n2 days ago\n•\n4.49k\n•\n307\nQwen/Qwen3-VL-8B-Instruct\nUpdated\n9 days ago\n•\n216k\n•\n315\nkrea/krea-realtime-video\nUpdated\n4 days ago\n•\n894\n•\n157\nBrowse 1M+ models\nSpaces\nRunning\n502\n502\nveo3.1-fast\

In [42]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [43]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 9 relevant links


# Welcome to Hugging Face – Where AI Gets a Hug! 🤗

## Who Are We?  
Picture a buzzing hive where AI enthusiasts, data scientists, and machine learning wizards join forces to build the future — and sprinkle a little fun along the way. That’s Hugging Face! We are the **AI community that’s not afraid to get up close and personal with models, datasets, and apps** — building an open, ethical AI future *together*.

From text and images to videos, audio, and even 3D, our platform hosts over **1 million models** and counting, plus more than **250,000 datasets**. Whether you want to train, share, or explore, we’re the hub you’ll want to be a part of.

*(Fun fact: we’re the AI equivalent of that cool neighborhood hangout where everyone knows your username, not just your name!)*

## Our Playground: The Hugging Face Platform  
- **Models Galore:** Browse, discover, and improve models from OCRs that read your handwriting (better than your grandma!) to real-time video generators.  
- **Datasets:** Dive into hundreds of thousands of datasets curated by pros and hobbyists alike.  
- **Spaces:** Launch and run AI-powered applications in our sandbox—like magic, but backed by open source.  
- **Community:** Collaborate, share your greatness, and build your machine learning portfolio.

We believe in **open collaboration, rapid progress**, and making machine learning accessible and delightful for everyone!

## Culture: Serious About AI, But Not About Boring  
- We're a global crew of AI dreamers who believe **collaboration beats competition**.  
- We strive for an **open and ethical AI future**—because the future is brighter when shared.  
- We keep things **light, witty, and supportive** — no techno-babble-only zones here.  
- Big ideas, big laughs, and a community-first mentality. Consider this your AI family reunion, minus the awkward small talk.

## Customers & Partners Who Hug Back  
From solo machine learning enthusiasts and independent researchers to startups and powerhouse enterprises, our users find value by:  
- Accelerating AI development with our **Open Source Stack**  
- Hosting their AI projects on a secure, scalable Enterprise platform  
- Getting dedicated support, fine-grained access controls, and **peace of mind** for team collaboration  
- Paying as they grow, with plans starting at just **$20/user/month** — because innovation shouldn’t break the bank.  

## Careers: Join the Hug!  
Looking for a career where you can:  
- Work on the bleeding edge of AI technology  
- Collaborate with some of the smartest brains in ML  
- Help grow a vibrant, open-source community dedicated to ethical AI  
- Enjoy a culture as welcoming as a warm, fuzzy emoji?

Hugging Face might just be your next home. Whether you’re a machine learning engineer, researcher, community builder, or developer advocate, we’re eager to meet you.

## Why Hugging Face?  
Because AI should be more than a buzzword — it should be a community effort filled with creativity, fun, and shared progress. We’re shaping the future *with* you, one model and dataset at a time.

### Ready to get hugged by the future of AI?  
**Sign up at** [huggingface.co](https://huggingface.co) and join the millions who are already building tomorrow, today.

---

## Hugging Face in a Nutshell:  
- 1M+ models, 250k+ datasets, 400k+ applications  
- Multi-modality support: text, image, video, audio, 3D, and beyond  
- Open Source & Enterprise solutions with scalable compute  
- A vibrant community that’s building AI for good—*and having fun doing it*  

---

*Hugging Face — Because Artificial Intelligence deserves a warm embrace.* 🤗

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [44]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [45]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Found 7 relevant links


# Hugging Face - Your AI Best Friend (And Then Some)

Welcome to **Hugging Face**, the AI community that's not just building the future—they’re *hugging* it close! If you think machine learning is all serious algorithms and labyrinthine math, think again. Here, innovation meets fun, collaboration, and boundary-pushing tech to create an open, ethical, and downright engaging AI ecosystem.

---

## What’s the Big Hug All About?

At its heart, Hugging Face is **the collaboration platform for the machine learning community**. It’s where engineers, scientists, developers, and curious minds like you converge to share over *1 million* models, hundreds of thousands of datasets, and thousands of interactive apps (they call these “Spaces”—and yes, they’re out of this world 🚀).

- **Models for Days:** Optical Character Recognition, video generation, vision transformers, you name it. Updated constantly by a vibrant global community.
- **Datasets Galore:** 250k+ datasets ready to fuel your next AI breakthrough.
- **Spaces:** Run your own apps or try others’ ML magic instantly.
- **Multimodal Mayhem:** Work with text, image, video, audio, even 3D — because one dimension is so 20th century.

---

## For the Cool People Who Use Hugging Face

- **Data Scientists & ML Engineers:** Build your portfolio, share your work, and learn in a community that’s as passionate as you.
- **Enterprises:** Looking for secure, scalable, and dedicated AI prowess? Teams can access enterprise-grade security and support, starting at just $20/user/month.
- **AI Enthusiasts & Students:** Learn, experiment, and get inspired with tons of free open source projects and documentation.
- **Developers:** Want to build better apps with AI? From models to acceleration, Hugging Face has you covered.

---

## Culture: More Than Just Models and Datasets

Sure, they build cutting-edge machine learning tools—but Hugging Face’s secret sauce is its **community vibe.**

- **Open and Ethical AI:** They believe AI should be built openly, responsibly, and with a big dollop of humanity.
- **Collaborative Spirit:** Share, experiment, and grow together.
- **Always Growing:** With a fast-growing community, your ideas won’t just stay ideas—they’ll evolve.
- **A Little Wit Added:** Because AI development is serious business but who says the process can’t have fun? (Ever seen a koala emoji as an app icon? Here, every day is quirky day 🐨)

---

## Career Opportunities: Ready to Join the Hug?

Are you passionate about AI and looking for a company pushing the bleeding edge of machine learning? Hugging Face is hiring! Though the careers page keeps things mysterious, rest assured they seek bright minds—engineers, ML researchers, product folks—to help build the open AI future.

Working here means:

- Joining a global **community of innovators**.
- Impacting open-source projects used by millions.
- Access to cutting-edge research and tooling.
- Being part of a company that’s as enthusiastic about ML ethics as it is about tech.

---

## Why Hugging Face? Because...

- They’re **not just a platform** — they’re the *home* of machine learning.
- They empower YOU to **accelerate your ML journey** whether you’re hacking solo or powering up your enterprise.
- With their **open source stack**, you’ll move faster than a T-Rex with a jetpack 🦖🚀.
- They combine **community + science + innovation** in a way that feels like the future *is* now.

---

# Ready to Hug?

Jump into the future with Hugging Face — where AI meets community, creativity, and a touch of quirky charm.

**Sign up today, explore 1M+ models, dive into datasets, or launch your own Space.**

Hugging Face: Because your AI journey deserves more than just code — it deserves a hug.

---

## Quick Links
- Explore AI Apps: [Visit Hugging Face](https://huggingface.co)
- Browse Models & Datasets
- Check Out Career Openings
- Join the Community on GitHub, Twitter, Discord

---

*P.S. No actual koalas were harmed in the making of this brochure, but they do have a knack for appearing in apps.* 🐨✨

In [46]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 15 relevant links


# Hugging Face: Your Friendly Neighborhood AI Sidekick 🤗

---

### Who We Are  
Welcome to **Hugging Face**, the ultimate hangout for AI enthusiasts, machine learning wizards, and data dreamers! We're not just building AI models; we're building **the future** — one dataset, one model, and one cool AI app at a time. Whether you're here to create, discover, collaborate, or just stare in awe at over **1 million models**, you've found your AI home.

---

### What We Offer: AI Buffet for Every Taste  
- **Models Galore:** From the latest OCR tech to video generation wizards, explore trending models updated fresh almost daily.  
- **Datasets Mountains:** Dig into over 250k datasets because your AI deserves *all* the knowledge.  
- **Spaces:** Build, run, and showcase your AI apps in style — hundreds of creative demos and new ones sprouting every minute!  
- **Community:** Join our global tribe where sharing, learning, and innovating is just everyday talk.

---

### Culture: Work Hard, Hug Harder  
At Hugging Face, we like our AI serious, but our vibes fun. Think quirky code jams, passionate debates over the best transformer model, and a culture that’s as inclusive as our open-source spirit. Whether you're a curious newbie or a seasoned engineer, there’s a place for you here to **grow, teach, and maybe even dance (virtual break rooms included).**

---

### Careers: Join the AI Revolution  
Want to make machines smarter and humans happier? We’re always on the lookout for passionate minds to join our **ever-expanding team**. From research magicians to backend sorcerers, check our careers page and see if you can snag a spot in the coolest AI co-working space on the planet!  

*Hint: If you like open collaboration, cutting-edge tech, and making a tangible impact—consider this your sign.*

---

### Enterprise: AI, But Make It Boss-Level  
Hugging Face isn’t just for the lone coder in a basement. Got a whole team? We’ve got you covered with **Team & Enterprise solutions** featuring:  
- Enterprise-grade security with **Single Sign-On (SSO)** for the fancy folks  
- Granular access controls so you control who sees what (think AI Fort Knox)  
- Advanced analytics & billing dashboards so you’re never lost in the AI wilderness  
- Priority support to keep your AI engines humming at peak performance  
- Extra storage and compute options to power your AI dreams without breaking the bank  

Pricing starts at $20/user/month – because powering the future shouldn’t empty your wallet.

---

### Prices That Play Nice  
Whether you’re a solo AI hero or running a squad of thousands:  
- **Pro Account ($9/month):** Amp your personal AI playground with more storage, credits, and all the fancy badges.  
- **Team Subscriptions:** Easy setup and scalable options for growing squads making a difference.  
- **Enterprise Plans:** Custom contracts for the big players who want serious performance and security.

---

### Why Hugging Face?  
- A **global AI party** where anyone can join, share & innovate  
- Tools and resources to build **your portfolio** and get noticed  
- Access to state-of-the-art open-source stacks that make complex AI simple  
- A thriving community ready to support, challenge, and celebrate your victories  

Come for the AI, stay for the hugs! 🤗

---

### Ready to Dive In?  
- Explore our universe at [huggingface.co](https://huggingface.co)  
- Start building, learning, or just marveling at the AI wonders  
- Join the future builders today and let's make smart machines together!

---

*Hugging Face: where AI meets humanity—with a warm, fuzzy twist.*

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>